In [77]:
### imports 
from pathlib import Path, PurePath
import os
import regex
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import ElasticsearchRetriever
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import TransformersReader
import numpy as np 
import json

In [82]:
from haystack.utils import print_answers

### splitting bills into individual articles 

In [2]:
prefix="/Users/filip/Documents/II_semestr/NLP/"
files = Path(prefix, "ustawy_cleaned_capitalized/").glob("*")

In [3]:
#test_f = next(files)
test_f = Path(prefix, "ustawy_cleaned_capitalized/", "1993_599.txt")

In [19]:
## prepare regex
#article_regex=regex.compile("\s*(Art\.\s+\d+\.)\s*((.|\n)*?(?=\s*(?:Art\.\s+\d+\.)\s*))")
article_regex=regex.compile("\s*(Art\.\s+\d+\.\\n)((.|\n)*?(?=\s*(?:(Art\.\s+\d+\.\\n|\Z))))")

In [38]:
separate_articles_dir = Path(prefix,"ustawy_osobne_artykuły")
os.makedirs(separate_articles_dir, exist_ok=True)

os.chdir(separate_articles_dir)
written_articles = []
for file in files:
    try: 
        with open(file, "r") as ff:
            name = file.stem
            cnt = ff.read()
            findings = regex.finditer(article_regex, cnt)
            for match in findings:
                article_name = f"{name}_{match[1].strip()}txt"  # each article ends with \. nontheless
                if article_name not in written_articles:
                    written_articles.append(article_name)
                    with open(article_name, "w") as ff2:
                        ff2.write(match[2])
                else:
                    print(f"zdublowany artykuł {article_name}")
    except UnicodeDecodeError as e:
        print(f"problem opening file: {file}")
        print(e)

zdublowany artykuł 2003_2256_Art. 3.txt
zdublowany artykuł 2003_2256_Art. 13.txt
zdublowany artykuł 2003_2256_Art. 18.txt
zdublowany artykuł 2003_1953_Art. 21.txt
zdublowany artykuł 2004_630_Art. 13.txt
zdublowany artykuł 2004_1963_Art. 30.txt


### Select passages that do not include any amendments 

In [56]:
amendments_patt = regex.compile("\bwprowadza\s+się\s+następujące\s+zmiany\b", regex.UNICODE)

In [57]:
separate_articles_dir = Path(prefix,"ustawy_osobne_artykuły").glob("*")
valid_articles = []
count = 1

for file in separate_articles_dir: 
    if count == 100:
        break
    with open(file, "r") as ff:
        if not regex.search(amendments_patt, ff.read()):
            print(f"file with no amendments: {ff.name}")
            valid_articles.append(ff.name)
            count += 1

file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2001_1229_Art. 36.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1999_802_Art. 9.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1995_515_Art. 7.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1996_56_Art. 4.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1997_781_Art. 38.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1996_465_Art. 580.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1997_753_Art. 56.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2003_2081_Art. 15.txt
file with no amendments: /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2004_285_Art. 35.txt
file with no amendments: /Users/filip/

In [63]:
## random 30 sample
articles_30 = np.random.choice(valid_articles, size=30, replace=False)
for i in articles_30:
    print(i)

/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1997_117_Art. 163.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2001_298_Art. 31.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1997_553_Art. 282.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1999_930_Art. 118.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2001_733_Art. 6.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1999_802_Art. 9.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2004_42_Art. 27.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2000_1250_Art. 43.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1994_592_Art. 9.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2004_883_Art. 1.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/2001_1545_Art. 299.txt
/Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/1997_739_Art. 100.txt
/Users/filip/D

30

### configure ES document store

In [47]:
articles_iter = separate_articles_dir.glob("*")
es_documents = []
for art in articles_iter:
    with open(art, "r") as f:
        try:
            es_documents.append({'content': f.read(), 'meta': {'name': art.stem}})
        except UnicodeDecodeError as e:
            print(f"cannot read {art}, problem is:\n{e}")

cannot read /Users/filip/Documents/II_semestr/NLP/ustawy_osobne_artykuły/.DS_Store, problem is:
'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte


In [53]:
# DocumentStore: holds all your data
document_store = ElasticsearchDocumentStore(analyzer="morfologik")

document_store.write_documents(es_documents)

In [92]:
### ask questions 
retriever = ElasticsearchRetriever(document_store)
#model_name = "deepset/roberta-base-squad2"
model_name = "enelpol/czywiesz-question"
reader = TransformersReader(model_name, use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at enelpol/czywiesz-question and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
pipe = ExtractiveQAPipeline(reader, retriever)

# Voilà! Ask a question!
question = "Jakie są organy administracji rybołóstwa morskiego?"
prediction = pipe.run(query=question, 
                      params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}}
                     )

In [94]:
print_answers(prediction)


Query: Jakie są organy administracji rybołóstwa morskiego?
Answers:
[   <Answer {'answer': '', 'type': 'extractive', 'score': 0.5000334337829101, 'context': None, 'offsets_in_document': [{'start': 0, 'end': 0}], 'offsets_in_context': [{'start': 0, 'end': 0}], 'document_id': None, 'meta': {}}>,
    <Answer {'answer': 'Okręgowe inspektoraty rybołóstwa', 'type': 'extractive', 'score': 0.0010166644351556897, 'context': 'Okręgowe inspektoraty rybołóstwa morskiego przejmą od urzędów morskich, wraz z\naktami, prowadzenie spr', 'offsets_in_document': [{'start': 0, 'end': 32}], 'offsets_in_context': [{'start': 0, 'end': 32}], 'document_id': 'a71b85916b2ab0e031c1311f37544456', 'meta': {'name': '1999_778_Art. 18'}}>,
    <Answer {'answer': 'rybołówstwa', 'type': 'extractive', 'score': 0.0009194017038680613, 'context': 'ałania, które nie zostały\nzakończone do dnia likwidacji inspektoratów rybołówstwa morskiego.', 'offsets_in_document': [{'start': 196, 'end': 207}], 'offsets_in_context': [{'start

In [95]:
pipe = ExtractiveQAPipeline(reader, retriever)

# Voilà! Ask a question!
question = "Czy do aplikantów kuratorskich stosuje się przepis art. 16?"
prediction2 = pipe.run(query=question, 
                      params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}}
                     )

In [96]:
print_answers(prediction2)


Query: Czy do aplikantów kuratorskich stosuje się przepis art. 16?
Answers:
[   <Answer {'answer': '', 'type': 'extractive', 'score': 0.5000951933388801, 'context': None, 'offsets_in_document': [{'start': 0, 'end': 0}], 'offsets_in_context': [{'start': 0, 'end': 0}], 'document_id': None, 'meta': {}}>,
    <Answer {'answer': 'aplikantów', 'type': 'extractive', 'score': 0.0029589703772217035, 'context': 'Do aplikantów kuratorskich stosuje się odpowiednio przepisy art. 14 ust. 1 i 6,\nart', 'offsets_in_document': [{'start': 3, 'end': 13}], 'offsets_in_context': [{'start': 3, 'end': 13}], 'document_id': '7ddda21dec4c06210d6bf4e826ac6afb', 'meta': {'name': '2001_1071_Art. 77'}}>,
    <Answer {'answer': 'aplikantów kuratorskich stosuje się odpowiednio przepisy art', 'type': 'extractive', 'score': 0.0025881691835820675, 'context': 'Do aplikantów kuratorskich stosuje się odpowiednio przepisy art. 14 ust. 1 i 6,\nart. 19 i 20.', 'offsets_in_document': [{'start': 3, 'end': 63}], 'offsets_in_con

In [90]:
pipe = ExtractiveQAPipeline(reader, retriever)

# Voilà! Ask a question!
question = "Czy można odwołać się od decyzji wydawanych przez komendantów granicznych placówek kontrolnych Straży Granicznej?"
prediction3 = pipe.run(query=question, 
                      params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}}
                     )

/Users/filip/virtenv_8_2/lib/python3.8/site-packages/transformers/pipelines/question_answering.py:189: UserWarning: topk parameter is deprecated, use top_k instead
  warnings.warn("topk parameter is deprecated, use top_k instead", UserWarning)


In [91]:
print_answers(prediction3)


Query: Czy można odwołać się od decyzji wydawanych przez komendantów granicznych placówek kontrolnych Straży Granicznej?
Answers:
[   <Answer {'answer': '', 'type': 'extractive', 'score': 0.500011076149347, 'context': None, 'offsets_in_document': [{'start': 0, 'end': 0}], 'offsets_in_context': [{'start': 0, 'end': 0}], 'document_id': None, 'meta': {}}>,
    <Answer {'answer': 'względu na miejsce\n     pobytu cudzoziemca lub komendanci granicznych placówek kontrolnych\n     Straży Granicznej,', 'type': 'extractive', 'score': 0.0003339773684274405, 'context': 'Granicznej,\n   2) o unieważnieniu wizy wydają wojewodowie właściwi ze względu na miejsce\n     pobytu cudzoziemca lub komendanci granicznych placówek kontrolnych\n     Straży Granicznej,\n   3) o określeniu czasu pobytu, o którym mowa w art. 13 ust. 4, wyda', 'offsets_in_document': [{'start': 205, 'end': 319}], 'offsets_in_context': [{'start': 70, 'end': 184}], 'document_id': 'f86fde1fe025ed2656e9d5b15d453cc7', 'meta': {'name': '

In [99]:
pipe = ExtractiveQAPipeline(reader, retriever)

# Voilà! Ask a question!
question = "W jakim przypadku szef BOR przyznaje uposażenie oraz inne świadczenia przysługujące w czasie oddelegowania?"
prediction4 = pipe.run(query=question, 
                      params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 3}}
                     )